In [16]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import time

In [17]:
# Fixed Parameters
S0 = 100
K = 60
r = 0.02
q = 0.01
sig = 0.35
T = 1.0

In [18]:
# model under consideration
model = 'LogNormal'

In [19]:
def logNormal(S, r, q, sig, S0, T):
    
    f = np.exp(-0.5*((np.log(S/S0)-(r-q-sig**2/2)*T)/(sig*np.sqrt(T)))**2)/(sig*S*np.sqrt(2*np.pi*T))
    return f

In [20]:
# Black-Merton-Scholes calculator
def BS_d1(S, K, r, q, sigma, tau):
    ''' Computes d1 for the Black-Merton-Scholes formula '''
    d1 = 1.0*(np.log(1.0 * S/K) + (r - q + sigma**2/2) * tau) / (sigma * np.sqrt(tau))
    return d1

def BS_d2(S, K, r, q, sigma, tau):
    ''' Computes d2 for the Black-Merton-Scholes formula '''
    d2 = 1.0*(np.log(1.0 * S/K) + (r - q - sigma**2/2) * tau) / (sigma * np.sqrt(tau))
    return d2

def BS_price(type_option, S, K, r, q, sigma, T, t=0):
    ''' Computes the Black-Merton-Scholes price for a 'call' or 'put' option '''
    tau = T - t
    d1 = BS_d1(S, K, r, q, sigma, tau)
    d2 = BS_d2(S, K, r, q, sigma, tau)
    if type_option == 'call':
        price = S * np.exp(-q * tau) * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)
    elif type_option == 'put':
        price = K * np.exp(-r * tau) * norm.cdf(-d2) - S * np.exp(-q * tau) * norm.cdf(-d1) 
    return price

# auxiliary function for computing implied vol
def aux_imp_vol(sigma, P, type_option, S, K, r, q, T, t=0):
    return P - BS_price(type_option, S, K, r, q, sigma, T, t)

In [21]:
def evaluateIntegral(*args):
    
    r   = args[0]
    q   = args[1]
    S0  = args[2]
    K   = args[3]
    sig = args[4]
    T   = args[5]
    N   = args[6]
    dS  = args[7]
    
    #discount factor
    df = np.exp(-r*T)
    
    #======================================================
    # evaluation of the integral using Trapezoidal method
    
    # Calls & Puts are different 
    #------------------------------------------------------
    
    # Call -- integrate from K to B
    sumC = 0
    S = np.zeros((N,1))
    for j in range(N):
        S[j] = K+j*dS    
    tmp = logNormal(S, r, q, sig, S0, T)
    for j in range(N):
        if j == 0:
            wj = dS/2
        else:
            wj = dS
        sumC += (S[j]-K)*tmp[j]*wj
    c0_KT = df * sumC
    
    # Put - integrate from 0 to K
    sumP = 0
    S = np.zeros((N,1))
    eta = (K-0.1)/N
    #print(eta)
    for j in range(N):
        S[j] = 0.1+j*eta
    tmp = logNormal(S, r, q, sig, S0, T)
    for j in range(N):
        if j == 0:
            wj = eta/2
        else:
            wj = eta
        sumP += (K-S[j])*tmp[j]*wj
    p0_KT = df * sumP
    
    return p0_KT, c0_KT 

In [23]:
#step-size
dS = 1

# number of grid points
n = 12
N = 2**n
B = K + N*dS

start_time = time.time()
print(' ')
print('===================')
print('Model is %s' % model)
print('-------------------')
arg = (r, q, S0, K, sig, T, N, dS)
p0_KT, c0_KT = evaluateIntegral(*arg)

put = BS_price('put', S0, K, r, q, sig, T)
call = BS_price('call', S0, K, r, q, sig, T)
print('')
print('B-M-S prices')
print('-----------------------------------------')
print("put price: ", put, " call price: ", call)
print('')
print('Prices via integration')
print('-----------------------------------------')
print("put price: ", p0_KT, " call price: ", c0_KT)
elapsed_time = time.time() - start_time
print('')
print("elapsed time: ", elapsed_time)

 
Model is LogNormal
-------------------

B-M-S prices
-----------------------------------------
put price:  0.7954007162235186  call price:  40.988463692734996

Prices via integration
-----------------------------------------
put price:  [0.79540058]  call price:  [40.98780838]

elapsed time:  0.03797340393066406
